In [1]:
%load_ext autoreload
%autoreload 2

## Import necessary modules
import os,sys
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator, AutoLocator, FormatStrFormatter, ScalarFormatter
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
from itertools import tee
from traitlets import traitlets

sys.path.append(os.path.abspath('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/xls_testruns/lib/'))
from ce_funclib import determine_kernzeit as dtkz
from ce_funclib import continuity_check, decminutes_to_mmss


from ipywidgets import widgets, interact, interactive, fixed, interact_manual, Layout
from IPython.display import display
#%matplotlib inline
%matplotlib tk


## Import data frome pickle generated from muß ein file mit agentenstats sein, wird generiert von Datensammler_Agenten.ipynb
pklpth='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/datenhalde/datapickles/Agenten-2018-04-21'

## Beispiel-Excel-Dateien mit und ohne Anrufe
emptyday='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2018-03-11.xls'
regularday='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2018-03-12.xls'


## Funktionen

In [2]:
def add_averages(frame):

    frame['acw_float']=frame['ht_float']-frame['tt_float']
    frame['avtt']=frame['tt_float']/frame['be']
    frame['avht']=frame['ht_float']/frame['be']
    frame['avacw']=frame['acw_float']/frame['be']
    
    colorder_with_abs=['an','be','vl','avht','avtt','avacw','ht_float','tt_float','acw_float']
    frame_with_abs=frame[colorder_with_abs]
    
    colorder_av_only=['an','be','vl','avht','avtt','avacw']
    frame_only_av=frame[colorder_av_only].fillna(0).unstack('bz')  # die Bearbeitungszeit soll hoch in die Spalten
    
    return frame_only_av

In [3]:
### dezimal-zeiten zu uhrzeiten umwandeln, inplace, daher noch eine kopie machen, um die dez.-daten zu erhalten...
def frame_dezimalzeiten_zu_mmss(frame):
    mit_mmss=frame.copy()
    lev=mit_mmss.columns.get_level_values(1)
    mit_mmss.iloc[:,(lev == 'avht') | (lev == 'avtt') | (lev == 'avacw')] = mit_mmss.iloc[:,(lev == 'avht') | (lev == 'avtt') | (lev == 'avacw')].applymap(decminutes_to_mmss)
    return mit_mmss
#frame_dezimalzeiten_zu_mmss(mit_summe_sortiert)

## Daten und Dataframe

In [4]:
big_frame=pd.read_pickle(pklpth)  # wo kommt der pickle her und wie generiere ich einen aktualisierten? -> Datensammler.py
big_frame['calweeks']=big_frame.index.strftime('%Y-%V')

In [5]:
sumfunx={'an':'sum','be':'sum','vl':'sum','ht_float':'sum','tt_float':'sum'}
colorder=['an','be','vl','tt_float','ht_float']

gruppiert=big_frame.groupby([pd.Grouper('calweeks'), pd.Grouper('agent'), pd.Grouper('ort'), pd.Grouper('bz')]).agg(sumfunx)
gruppiert = gruppiert.unstack('bz').fillna(0).stack('bz') # fehlene Bearbeitungseiten mit 0 auffüllen
k_n = gruppiert.sum(level=['calweeks','agent','ort']) # einen Zwischenframe erstellen, der nur die Kern- und Nebenzeit summiert

mit_gesamt_bz=gruppiert.append(k_n.assign(bz='g').set_index('bz', append=True)).sort_index() # die Summe dem alten Frame beigeben

# https://stackoverflow.com/questions/17287933/filling-in-date-gaps-in-multiindex-pandas-dataframe
# https://stackoverflow.com/questions/20110170/turn-pandas-multi-index-into-column
# https://stackoverflow.com/questions/45856724/how-to-get-sub-total-according-to-index-level-in-multi-index-pandas-dataframe

In [6]:
## Summe für jede Woche als Agent "SUM" hinzufügen

summenzeile=mit_gesamt_bz.sum(level=['calweeks','bz']) # Zwischenframe, der die Summe für jede Woche beinhaltet
sume=summenzeile.assign(agent='SUM',ort='N').set_index(['agent','ort'], append=True).reorder_levels(['calweeks','agent','ort','bz'], axis=0)
mit_summe=mit_gesamt_bz.append(sume).sort_index() # Summe wird eingefuegt und mit Name und 'Ort' versehen

In [7]:
mit_durchschnitt=add_averages(mit_summe)

In [8]:
mit_durchschnitt=mit_durchschnitt.swaplevel(0,1,1).sort_index(1) # Angeordnet nach Bearbeitungszeit -> Anrufdaten
mit_durchschnitt.columns.names=['bz','daten'] # Namen fuer die Spalten-Indexlevels vergeben
# https://stackoverflow.com/questions/40411300/swapping-ordering-multi-index-columns-in-pandas

In [9]:
l0_order=['g','k','n']                        
l1_order=['an','be','vl','avht','avtt','avacw']

mit_durchschnitt=mit_durchschnitt.reindex_axis(l0_order, axis=1, level=0).reindex_axis(l1_order, axis=1, level=1)   # Reihenfolge der MultiindexSpalten

In [10]:
# Sortieren nach Kalenderwoche und innerhalb derer nach Anzahl angenommener Calls
mit_durchschnitt=mit_durchschnitt.sort_values(('g','an'), ascending=False).sort_index(level='calweeks', sort_remaining=False)
#mit_durchschnitt.head(15)
# https://stackoverflow.com/questions/43472125/sort-pandas-dataframe-with-multiindex-according-to-column-value

#### mit_durchschnitt = selbes Format wie Exceltabelle

## Styles und Plots

In [11]:
#def cfun(x):
#    #print(type(x), end=' ')
#    #print(x.name)
#    if x.name[0] == 'ttl':
#        return ['background-color: blue']
#    elif x.name[0] == 'k':
#        return ['background-color: red']
#    elif x.name[0] == 'n':
#        return ['background-color: yellow']
#    else:
#        return['']
#def dfun(x):
#    is_sum = x.index.get_level_values('agent') == 'SUMME'
#    return ['color: red' if v else '' for v in is_sum]


#colored=mit_mmss.style.apply(cfun).apply(dfun)#.format(styledict)

In [12]:
styledict={('g','avht'):'{:.2f}',('g','avtt'):'{:.2f}',('g','avacw'):'{:.2f}',\
                                     ('k','avht'):'{:.2f}',('k','avtt'):'{:.2f}',('k','avacw'):'{:.2f}',\
                                     ('n','avht'):'{:.2f}',('n','avtt'):'{:.2f}',('n','avacw'):'{:.2f}',}